In [ ]:
from datasets import load_from_disk
from IPython.display import display
import matplotlib.pyplot as plt

# 데이터셋 로드
dataset_path = "merged_dataset"  
ds = load_from_disk(dataset_path)

print(" Dataset 구조 확인 (index, is_same, image)")

for i in range(10000):
    sample = ds[i]
    print(f"[{i}] index: {sample['index']}, is_same: {sample['is_same']}")
    
    # 이미지 시각화
    plt.imshow(sample['image'])
    plt.title(f"index: {sample['index']}, is_same: {sample['is_same']}")
    plt.axis('off')
    plt.show()


In [ ]:
from datasets import load_from_disk
from collections import Counter
import matplotlib.pyplot as plt

# Hugging Face Dataset 경로 지정
dataset_path = "Asian_celebrity_LFW_huggingface"  

# Dataset 불러오기
ds = load_from_disk(dataset_path)

# is_same 분포 확인
is_same_values = [int(sample['is_same']) for sample in ds]
counter = Counter(is_same_values)

num_true = counter[1]
num_false = counter[0]
total = num_true + num_false

print(" is_same 분포")
print(f" - is_same=True  (same person):   {num_true}")
print(f" - is_same=False (diff person):   {num_false}")
print(f" - 총 쌍 수: {total}")
print(f" - 비율: True {num_true/total:.2%} / False {num_false/total:.2%}")

# 시각화 (pie chart)
plt.figure(figsize=(5, 5))
plt.pie(
    [num_true, num_false],
    labels=['is_same=True', 'is_same=False'],
    autopct='%1.1f%%',
    startangle=90
)
plt.title("is_same 분포 (same vs. diff)")
plt.axis('equal')
plt.show()


In [ ]:
from datasets import load_from_disk

# 데이터셋 경로
dataset_path = "lfw"  
ds = load_from_disk(dataset_path)

# 총 개수 확인
num_samples = len(ds)

# 실제로 이미지 타입이 들어 있는지, 누락된 건 없는지 확인
num_valid_images = sum(1 for sample in ds if sample['image'] is not None)

print(f"총 샘플 수 (image, is_same 포함): {num_samples}")
print(f"실제 image가 정상적으로 들어간 개수: {num_valid_images}")


In [ ]:
from datasets import load_from_disk
from collections import Counter

dataset_path = "lfw"
ds = load_from_disk(dataset_path)

num_folds = 10
samples_per_fold = len(ds) // num_folds

print("Fold별 is_same=True 개수 확인")

for i in range(num_folds):
    start = i * samples_per_fold
    end = start + samples_per_fold
    fold = ds.select(range(start, end))  # select로 자르기

    is_same_list = [int(fold[i]['is_same']) for i in range(len(fold))]  # 올바르게 인덱싱
    count = Counter(is_same_list)
    print(f"Fold {i}: is_same=True {count[1]}, is_same=False {count[0]}")



In [ ]:
from datasets import load_from_disk

# 1. 데이터 로드
ds = load_from_disk("Asian_celebrity_LFW_huggingface")

# 2. 쌍 단위로 확인
print("동일 이미지가 두 장인 쌍 탐색")
duplicate_pairs = 0
for i in range(0, len(ds), 2):
    img1 = ds[i]['image']
    img2 = ds[i+1]['image']
    
    if list(img1.getdata()) == list(img2.getdata()):  # 픽셀 단위 비교
        print(f"같은 이미지가 쌍으로 들어간 샘플: index {i} & {i+1}, is_same={ds[i]['is_same']}")
        duplicate_pairs += 1

print(f"총 중복 이미지 쌍 개수: {duplicate_pairs}")


In [ ]:
from datasets import load_from_disk
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# 🔹 1. 데이터 로드
dataset_path = "Asian_celebrity_LFW_huggingface"  
ds = load_from_disk(dataset_path)

# 🔹 2. 타입 확인
print("타입 확인")
print("type(index):", type(ds[0]["index"]))
print("type(is_same):", type(ds[0]["is_same"]))
print("index 예시:", ds[0]["index"])
print("is_same 예시:", ds[0]["is_same"])

# 🔹 3. 라벨 통계
is_same_list = [bool(ds[i]["is_same"]) for i in range(len(ds))]
label_counts = Counter(is_same_list)
print("\nis_same 분포")
print("is_same=True (same person):", label_counts[True])
print("is_same=False (diff person):", label_counts[False])
print("총 샘플 수:", len(ds))

# 🔹 4. index 정렬 여부 확인
indices = [ds[i]["index"] for i in range(len(ds))]
is_sorted = indices == sorted(indices)
print("\nindex 정렬 상태:", "정렬됨" if is_sorted else "정렬 안됨")

# 🔹 5. 이미지 + 라벨 시각화 (처음 6쌍)
print("\n샘플 이미지 시각화")
for i in range(0, 12, 2):
    img1 = ds[i]['image']
    img2 = ds[i+1]['image']
    label = ds[i]['is_same']

    fig, axs = plt.subplots(1, 2, figsize=(6, 3))
    axs[0].imshow(img1)
    axs[0].axis('off')
    axs[0].set_title(f"index: {ds[i]['index']}")

    axs[1].imshow(img2)
    axs[1].axis('off')
    axs[1].set_title(f"index: {ds[i+1]['index']}")

    plt.suptitle(f"Pair {i//2} - is_same: {label}")
    plt.tight_layout()
    plt.show()


In [ ]:
from datasets import load_from_disk
import os

# 1. 원본 데이터셋 경로 입력
original_dataset_path = "Asian_celebrity_LFW_huggingface" 

# 2. 저장할 정렬된 데이터셋 경로
sorted_dataset_path = original_dataset_path + "_sorted"
os.makedirs(sorted_dataset_path, exist_ok=True)

# 3. 데이터셋 로드
print(f"원본 데이터셋 로딩 중: {original_dataset_path}")
ds = load_from_disk(original_dataset_path)

# 4. index 기준 정렬
print("index 기준 정렬 중...")
ds_sorted = ds.sort("index")

# 5. 저장
print(f" 정렬된 데이터셋 저장 중: {sorted_dataset_path}")
ds_sorted.save_to_disk(sorted_dataset_path)

print("정렬 및 저장 완료!")
